# Interactive Tutorial for the HWO High-Resolution Imager (HRI) 

This notebook shows how to use the basic capabilities of [SYOTools](https://github.com/tumlinson/luvoir_simtools) for ETCs. It uses Bokeh & astropy to visualize exposure time calculators (ETCs), as well as other science cases, as a function of various observatory parameters. While SYOTools can be used independently of a particular observatory or science case, it has been primarily written to facilitate the design of the [Habitable Worlds Observatory](https://www.habitableworldsobservatory.org).

SYOTools is divided into two main parts, contained in the `syotools.models` and `syotools.interface` subpackages. This Jupyter Notebook is intended as a walkthrough for using `syotools.models` to perform calculations. A future tutorial on using `syotools.interface` to design a web tool is in preparation; however, it will not be a similar Jupyter Notebook, as the interface framework is highly integrated with Bokeh Server. Instead, we will use Bokeh's `output_notebook` function, along with IPython interactors (as described [here](https://github.com/bokeh/bokeh/blob/0.12.13/examples/howto/notebook_comms/Jupyter%20Interactors.ipynb)), to visualize the example calculations below. 



In [ ]:
#Import Bokeh interface tools
from ipywidgets import interact
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import column
output_notebook()

#Import numpy, syotools, and astropy.units
### NOTE: you will likely receive some pysynphot warnings when executing these imports, these are expected
import numpy as np
from syotools import cdbs #make sure that pysynphot can find its reference files!
from syotools.models import Telescope, Camera, Spectrograph #models for the observatory and instruments
from syotools.utils.jsonunit import str_jsunit #for printing JsonUnit and JsonSpectrum wrappers in a readable way
from syotools.utils import pre_encode, pre_decode
from syotools.spectra import SpectralLibrary
import astropy.units as u

## Example 1: HRI ETC

*"The High Resolution Imager (HRI) instrument is the primary astronomical imaging instrument for observations in the near UV through the near IR. The hri design provides a 2 x 3 arcminute field-of-view, taking full advantage of the angular resolution provided by the telescope, and consists of two channels - an ultraviolet-visible (UVIS) channel covering 200 nm - 950 nm and a near-infrared (NIR) channel covering the range 800 nm - 2200 nm. The respective focal plane detector arrays provide Nyquist sampled images at 400 nm (2.73 mas/pixel) for UVIS imaging and at 1200 nm (8.20 mas/pixel) for NIR imaging."* (adapted from LUVOIR STDT study) 

In this example, we will create a exposure time calculator for HRI, so that we can calculate the signal-to-noise ratio (SNR) for several possible template spectra. This approximates some of the functionality of [the official HRI ETC tool](http://hwo.stsci.edu/camera_etc).

In [ ]:
#Instatiate the observatory using syotools.models.Telescope
#This loads the default Telescope values, which are already based on the LUVOIR design
luvoir_ex1 = Telescope()
luvoir_ex1.set_from_json('EAC1') 

In [ ]:
#Instantiate the instrument, using syotools.models.Camera, and link it with the telescope
#This loads the default Camera values, which are already based on the LUVOIR-HRI design
hri = Camera()
luvoir_ex1.add_camera(hri)

In [ ]:
#Create a new photometric exposure for the camera
hri_exposure = hri.create_exposure()

#Print the available wave bands
pivotwave, bandpass = hri.recover('pivotwave', 'derived_bandpass')

print("HRI wave bands:") 
for band, pwave, bpass in zip(hri.bandnames, pivotwave, bandpass):
    print(u"   {:3s} - {:5.2f} ± {:5.2f}".format(band, pwave, bpass)) 

#Print the default template
default_hri_template = SpectralLibrary[hri_exposure.sed_id]
print("Current SED template: {}".format(default_hri_template)) 

hri_template_codes = ['fab', 'o5v', 'b5v', 'g2v', 'm2v', 'orion', 'elliptical', 'sbc', 'starburst', 'ngc1068']
available_hri_templates = [SpectralLibrary[tc] for tc in hri_template_codes]

In [ ]:
#Create the Bokeh SED figure
hri_sed = hri_exposure.recover('sed')
hri_sed.convert('abmag')
hri_sed.convert('nm')
hri_sed_fig = figure(height=300, width=600, title="SED", x_axis_label="Wavelength [nm]",
                 y_axis_label="AB Mag", y_range=(35, 21), x_range=(120, 2300))
hri_sed_line = hri_sed_fig.line(hri_sed.wave, hri_sed.flux, color='orange', line_width=3)

In [ ]:
#Create the Bokeh SNR figure
hri_snr = hri_exposure.recover('snr')
hri_snr_fig = figure(height=300, width=600, title="SNR", x_axis_label="Wavelength [nm]", 
                 y_axis_label="SNR", y_range=(0, 20), x_range=(120, 2300))
uv_hri_snr = hri_snr_fig.line(pivotwave[:2].value, hri_snr[:2].value, color='orange', line_width=3)
vis_hri_snr = hri_snr_fig.line(pivotwave[2:-3].value, hri_snr[2:-3].value, color='blue', line_width=3)
ir_hri_snr = hri_snr_fig.line(pivotwave[-3:].value, hri_snr[-3:].value, color='red', line_width=3)

In [ ]:
#Define the update callback function for interactive inputs
def hri_update(template=default_hri_template, aperture=8., exptime=1., v_magnitude=30.):
    #find the correct template code
    sed_id = hri_template_codes[available_hri_templates.index(template)]
    
    #turn off calculations until everything is updated
    hri_exposure.disable()
    
    #update all of the telescope & exposure parameters
    hri_exposure.exptime = exptime * u.h
    hri_exposure.sed_id = sed_id
    luvoir_ex1.aperture = aperture * u.m
    hri_exposure.renorm_sed(v_magnitude * u.ABmag)
    
    #turn calculations back on and recalculate based on updated parameters
    hri_exposure.enable()
    
    #recover the recalculated values, and make sure everything is in the right units
    hri_sed, hri_snr = hri_exposure.recover('sed', 'snr')
    hri_sed.convert('nm')
    hri_sed.convert('abmag')
    
    #sanitize the sed fluxes because some of the pysynphot spectra don't play nice
    hri_flux = hri_sed.flux
    hri_flux[~np.isfinite(hri_flux)] = v_magnitude
    
    #update the SED figure
    hri_sed_fig.y_range.start = hri_flux.max() + 5.
    hri_sed_fig.y_range.end = hri_flux.min() - 5.
    hri_sed_line.data_source.data = {'x': hri_sed.wave, 'y': hri_flux}
    
    #update the SNR figure
    hri_snr_fig.y_range.start = 0.
    hri_snr_fig.y_range.end = max(1.3 * hri_snr.value.max(), 5.)
    uv_hri_snr.data_source.data['y'] = hri_snr.value[:2]
    vis_hri_snr.data_source.data['y'] = hri_snr.value[2:-3]
    ir_hri_snr.data_source.data['y'] = hri_snr.value[-3:]
    
    #update the plots
    push_notebook()

In [ ]:
# Show the plots
# depending on your jupyter config you may need to shift+enter this cell 
# after changing the sliders below to see an updated result plot
hri_handle = show(column(hri_sed_fig, hri_snr_fig), notebook_handle=True)

In [ ]:
#Create the interactive inputs
#shift+enter here will reset to defaults. 
hri_inputs = interact(hri_update, template=available_hri_templates, aperture=(2.0, 12.0), exptime=(0.1, 10.0, 0.1), 
         v_magnitude=(20.0, 35.0, 0.1))